In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

schema = ['id','name','salary','dept','gender']

emp_df = spark.createDataFrame(data=emp_data, schema=schema)

In [0]:
emp_df.show()

+---+--------+------+---------+------+
| id|    name|salary|     dept|gender|
+---+--------+------+---------+------+
|  1|  manish| 50000|       IT|     m|
|  2|  vikash| 60000|    sales|     m|
|  3| raushan| 70000|marketing|     m|
|  4|  mukesh| 80000|       IT|     m|
|  5|   priti| 90000|    sales|     f|
|  6|  nikita| 45000|marketing|     f|
|  7|  ragini| 55000|marketing|     f|
|  8|   rashi|100000|       IT|     f|
|  9|  aditya| 65000|       IT|     m|
| 10|   rahul| 50000|marketing|     m|
| 11|   rakhi| 50000|       IT|     f|
| 12|akhilesh| 90000|    sales|     m|
+---+--------+------+---------+------+



### Group By

In [0]:
emp_df.groupBy("dept")\
    .agg(sum("salary")).show()

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|       IT|     345000|
|marketing|     220000|
|    sales|     240000|
+---------+-----------+



### Window Function

In [0]:
from pyspark.sql.window import Window

window = Window.partitionBy("dept")

emp_df.withColumn("total_salary",sum(col('salary')).over(window))\
    .show(truncate=False)

+---+--------+------+---------+------+------------+
|id |name    |salary|dept     |gender|total_salary|
+---+--------+------+---------+------+------------+
|1  |manish  |50000 |IT       |m     |345000      |
|4  |mukesh  |80000 |IT       |m     |345000      |
|8  |rashi   |100000|IT       |f     |345000      |
|9  |aditya  |65000 |IT       |m     |345000      |
|11 |rakhi   |50000 |IT       |f     |345000      |
|3  |raushan |70000 |marketing|m     |220000      |
|6  |nikita  |45000 |marketing|f     |220000      |
|7  |ragini  |55000 |marketing|f     |220000      |
|10 |rahul   |50000 |marketing|m     |220000      |
|2  |vikash  |60000 |sales    |m     |240000      |
|5  |priti   |90000 |sales    |f     |240000      |
|12 |akhilesh|90000 |sales    |m     |240000      |
+---+--------+------+---------+------+------------+



In [0]:
from pyspark.sql.window import Window

window = Window.partitionBy("dept").orderBy("salary")

emp_df.withColumn('row_number',row_number().over(window))\
    .show(truncate=False)

+---+--------+------+---------+------+----------+
|id |name    |salary|dept     |gender|row_number|
+---+--------+------+---------+------+----------+
|1  |manish  |50000 |IT       |m     |1         |
|11 |rakhi   |50000 |IT       |f     |2         |
|9  |aditya  |65000 |IT       |m     |3         |
|4  |mukesh  |80000 |IT       |m     |4         |
|8  |rashi   |100000|IT       |f     |5         |
|6  |nikita  |45000 |marketing|f     |1         |
|10 |rahul   |50000 |marketing|m     |2         |
|7  |ragini  |55000 |marketing|f     |3         |
|3  |raushan |70000 |marketing|m     |4         |
|2  |vikash  |60000 |sales    |m     |1         |
|5  |priti   |90000 |sales    |f     |2         |
|12 |akhilesh|90000 |sales    |m     |3         |
+---+--------+------+---------+------+----------+



In [0]:
from pyspark.sql.window import Window

window = Window.partitionBy("dept").orderBy("salary")

emp_df.withColumn("row_number",row_number().over(window))\
    .withColumn("rank",rank().over(window))\
    .withColumn("dence_rank",dense_rank().over(window))\
    .show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |salary|dept     |gender|row_number|rank|dence_rank|
+---+--------+------+---------+------+----------+----+----------+
|1  |manish  |50000 |IT       |m     |1         |1   |1         |
|11 |rakhi   |50000 |IT       |f     |2         |1   |1         |
|9  |aditya  |65000 |IT       |m     |3         |3   |2         |
|4  |mukesh  |80000 |IT       |m     |4         |4   |3         |
|8  |rashi   |100000|IT       |f     |5         |5   |4         |
|6  |nikita  |45000 |marketing|f     |1         |1   |1         |
|10 |rahul   |50000 |marketing|m     |2         |2   |2         |
|7  |ragini  |55000 |marketing|f     |3         |3   |3         |
|3  |raushan |70000 |marketing|m     |4         |4   |4         |
|2  |vikash  |60000 |sales    |m     |1         |1   |1         |
|5  |priti   |90000 |sales    |f     |2         |2   |2         |
|12 |akhilesh|90000 |sales    |m     |3         |2   |2         |
+---+-----

In [0]:
# top two earner on each dept and each gender

from pyspark.sql.window import Window

window = Window.partitionBy("dept","gender").orderBy("salary")

emp_df.withColumn("row_number",row_number().over(window))\
    .withColumn("rank",rank().over(window))\
    .withColumn("dence_rank",dense_rank().over(window))\
    .show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |salary|dept     |gender|row_number|rank|dence_rank|
+---+--------+------+---------+------+----------+----+----------+
|11 |rakhi   |50000 |IT       |f     |1         |1   |1         |
|8  |rashi   |100000|IT       |f     |2         |2   |2         |
|1  |manish  |50000 |IT       |m     |1         |1   |1         |
|9  |aditya  |65000 |IT       |m     |2         |2   |2         |
|4  |mukesh  |80000 |IT       |m     |3         |3   |3         |
|6  |nikita  |45000 |marketing|f     |1         |1   |1         |
|7  |ragini  |55000 |marketing|f     |2         |2   |2         |
|10 |rahul   |50000 |marketing|m     |1         |1   |1         |
|3  |raushan |70000 |marketing|m     |2         |2   |2         |
|5  |priti   |90000 |sales    |f     |1         |1   |1         |
|2  |vikash  |60000 |sales    |m     |1         |1   |1         |
|12 |akhilesh|90000 |sales    |m     |2         |2   |2         |
+---+-----

In [0]:
# top two performer in each department

from pyspark.sql.window import Window

window = Window.partitionBy("dept").orderBy(desc("salary"))

emp_df.withColumn("row_number",row_number().over(window))\
    .withColumn("rank",rank().over(window))\
    .withColumn("dense_rank",dense_rank().over(window))\
    .filter(col("dense_rank") <= 2)\
    .show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |salary|dept     |gender|row_number|rank|dense_rank|
+---+--------+------+---------+------+----------+----+----------+
|8  |rashi   |100000|IT       |f     |1         |1   |1         |
|4  |mukesh  |80000 |IT       |m     |2         |2   |2         |
|3  |raushan |70000 |marketing|m     |1         |1   |1         |
|7  |ragini  |55000 |marketing|f     |2         |2   |2         |
|5  |priti   |90000 |sales    |f     |1         |1   |1         |
|12 |akhilesh|90000 |sales    |m     |2         |1   |1         |
|2  |vikash  |60000 |sales    |m     |3         |3   |2         |
+---+--------+------+---------+------+----------+----+----------+



In [0]:
# top two performer in each department and each gender

from pyspark.sql.window import Window

window = Window.partitionBy("dept","gender").orderBy(desc("salary"))

emp_df.withColumn("row_number",row_number().over(window))\
    .withColumn("rank",rank().over(window))\
    .withColumn("dense_rank",dense_rank().over(window))\
    .filter(col("dense_rank") <= 2)\
    .show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |salary|dept     |gender|row_number|rank|dense_rank|
+---+--------+------+---------+------+----------+----+----------+
|8  |rashi   |100000|IT       |f     |1         |1   |1         |
|11 |rakhi   |50000 |IT       |f     |2         |2   |2         |
|4  |mukesh  |80000 |IT       |m     |1         |1   |1         |
|9  |aditya  |65000 |IT       |m     |2         |2   |2         |
|7  |ragini  |55000 |marketing|f     |1         |1   |1         |
|6  |nikita  |45000 |marketing|f     |2         |2   |2         |
|3  |raushan |70000 |marketing|m     |1         |1   |1         |
|10 |rahul   |50000 |marketing|m     |2         |2   |2         |
|5  |priti   |90000 |sales    |f     |1         |1   |1         |
|12 |akhilesh|90000 |sales    |m     |1         |1   |1         |
|2  |vikash  |60000 |sales    |m     |2         |2   |2         |
+---+--------+------+---------+------+----------+----+----------+



# lead and lag in spark | window function in pyspark

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema = ["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data = product_data, schema = product_schema)

In [0]:
product_df.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         1|      iphone|01-01-2023|1500000|
|         2|     samsung|01-01-2023|1100000|
|         3|     oneplus|01-01-2023|1100000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



In [0]:
#lag - three argument - column , no. of previus records , if no record then null
window = Window.partitionBy('product_id').orderBy("sales_date")

last_month_df = product_df.withColumn("previous_month_sales",lag(col("sales"),1).over(window))

last_month_df.show()

+----------+------------+----------+-------+--------------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|
+----------+------------+----------+-------+--------------------+
|         1|      iphone|01-01-2023|1500000|                null|
|         1|      iphone|01-02-2023|1300000|             1500000|
|         1|      iphone|01-03-2023|1600000|             1300000|
|         1|      iphone|01-04-2023|1700000|             1600000|
|         1|      iphone|01-05-2023|1200000|             1700000|
|         1|      iphone|01-06-2023|1100000|             1200000|
|         2|     samsung|01-01-2023|1100000|                null|
|         2|     samsung|01-02-2023|1120000|             1100000|
|         2|     samsung|01-03-2023|1080000|             1120000|
|         2|     samsung|01-04-2023|1800000|             1080000|
|         2|     samsung|01-05-2023| 980000|             1800000|
|         2|     samsung|01-06-2023|1100000|              980000|
|         

In [0]:
#lead - three argument - column , no. of next records , if no record then null
window = Window.partitionBy('product_id').orderBy("sales_date")

next_month_df = product_df.withColumn("previous_month_sales",lead(col("sales"),1).over(window))

last_month_df.show()

+----------+------------+----------+-------+--------------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|
+----------+------------+----------+-------+--------------------+
|         1|      iphone|01-01-2023|1500000|                null|
|         1|      iphone|01-02-2023|1300000|             1500000|
|         1|      iphone|01-03-2023|1600000|             1300000|
|         1|      iphone|01-04-2023|1700000|             1600000|
|         1|      iphone|01-05-2023|1200000|             1700000|
|         1|      iphone|01-06-2023|1100000|             1200000|
|         2|     samsung|01-01-2023|1100000|                null|
|         2|     samsung|01-02-2023|1120000|             1100000|
|         2|     samsung|01-03-2023|1080000|             1120000|
|         2|     samsung|01-04-2023|1800000|             1080000|
|         2|     samsung|01-05-2023| 980000|             1800000|
|         2|     samsung|01-06-2023|1100000|              980000|
|         

In [0]:
last_month_df.withColumn('per_loss_gain',col("sales") - col("previous_month_sales")).show()

+----------+------------+----------+-------+--------------------+-------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|per_loss_gain|
+----------+------------+----------+-------+--------------------+-------------+
|         1|      iphone|01-01-2023|1500000|                null|         null|
|         1|      iphone|01-02-2023|1300000|             1500000|      -200000|
|         1|      iphone|01-03-2023|1600000|             1300000|       300000|
|         1|      iphone|01-04-2023|1700000|             1600000|       100000|
|         1|      iphone|01-05-2023|1200000|             1700000|      -500000|
|         1|      iphone|01-06-2023|1100000|             1200000|      -100000|
|         2|     samsung|01-01-2023|1100000|                null|         null|
|         2|     samsung|01-02-2023|1120000|             1100000|        20000|
|         2|     samsung|01-03-2023|1080000|             1120000|       -40000|
|         2|     samsung|01-04-2023|1800

In [0]:
last_month_df.withColumn('per_loss_gain',round(((col("sales") - col("previous_month_sales"))/col('sales'))*100,2)).show()

+----------+------------+----------+-------+--------------------+-------------+
|product_id|product_name|sales_date|  sales|previous_month_sales|per_loss_gain|
+----------+------------+----------+-------+--------------------+-------------+
|         1|      iphone|01-01-2023|1500000|                null|         null|
|         1|      iphone|01-02-2023|1300000|             1500000|       -15.38|
|         1|      iphone|01-03-2023|1600000|             1300000|        18.75|
|         1|      iphone|01-04-2023|1700000|             1600000|         5.88|
|         1|      iphone|01-05-2023|1200000|             1700000|       -41.67|
|         1|      iphone|01-06-2023|1100000|             1200000|        -9.09|
|         2|     samsung|01-01-2023|1100000|                null|         null|
|         2|     samsung|01-02-2023|1120000|             1100000|         1.79|
|         2|     samsung|01-03-2023|1080000|             1120000|         -3.7|
|         2|     samsung|01-04-2023|1800

In [0]:
#Q1. what is the %age of sales each month based on last 6 month sales.
window = Window.partitionBy("product_id")
last_six_month_df = product_df.withColumn("previous_six_month_total_sales", sum("sales").over(window))\
    .withColumn("perc_sales_each_month", round((col("sales")/col("previous_six_month_total_sales")) * 100, 2) )
last_six_month_df.show()

+----------+------------+----------+-------+------------------------------+---------------------+
|product_id|product_name|sales_date|  sales|previous_six_month_total_sales|perc_sales_each_month|
+----------+------------+----------+-------+------------------------------+---------------------+
|         1|      iphone|01-01-2023|1500000|                       8400000|                17.86|
|         1|      iphone|01-02-2023|1300000|                       8400000|                15.48|
|         1|      iphone|01-03-2023|1600000|                       8400000|                19.05|
|         1|      iphone|01-04-2023|1700000|                       8400000|                20.24|
|         1|      iphone|01-05-2023|1200000|                       8400000|                14.29|
|         1|      iphone|01-06-2023|1100000|                       8400000|                 13.1|
|         2|     samsung|01-01-2023|1100000|                       7180000|                15.32|
|         2|     sam

### First and Last in Spark

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *

In [0]:
# question - find out difference in sales of each product from their first month sales to latest sales?
product_data = [
(2,"samsung","01-01-1995",11000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-01-2006",15000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(3,"oneplus","01-01-2010",23000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]


product_schema = ['product_id','product_name','sales_date','sales']

product_df = spark.createDataFrame(data=product_data, schema=product_schema)

product_df.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         2|     samsung|01-01-1995|  11000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-01-2006|  15000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         3|     oneplus|01-01-2010|  23000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



### Unbounded preceding & Unbounded following

In [0]:
window = Window.partitionBy("product_id").orderBy("sales_date")\
    .rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

In [0]:
product_df.withColumn("first_sales",first("sales").over(window))\
    .withColumn("last_sales",last("sales").over(window)).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Window [product_id#828L, product_name#829, sales_date#830, sales#831L, first(sales#831L, false) windowspecdefinition(product_id#828L, sales_date#830 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS first_sales#854L, last(sales#831L, false) windowspecdefinition(product_id#828L, sales_date#830 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS last_sales#861L], [product_id#828L], [sales_date#830 ASC NULLS FIRST]
   +- Sort [product_id#828L ASC NULLS FIRST, sales_date#830 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(product_id#828L, 200), ENSURE_REQUIREMENTS, [plan_id=1053]
         +- Scan ExistingRDD[product_id#828L,product_name#829,sales_date#830,sales#831L]




In [0]:
product_df.withColumn("first_sales",first("sales").over(window))\
    .withColumn("last_sales",last("sales").over(window)).show()

+----------+------------+----------+-------+-----------+----------+
|product_id|product_name|sales_date|  sales|first_sales|last_sales|
+----------+------------+----------+-------+-----------+----------+
|         1|      iphone|01-01-2006|  15000|      15000|   1100000|
|         1|      iphone|01-02-2023|1300000|      15000|   1100000|
|         1|      iphone|01-03-2023|1600000|      15000|   1100000|
|         1|      iphone|01-04-2023|1700000|      15000|   1100000|
|         1|      iphone|01-05-2023|1200000|      15000|   1100000|
|         1|      iphone|01-06-2023|1100000|      15000|   1100000|
|         2|     samsung|01-01-1995|  11000|      11000|   1100000|
|         2|     samsung|01-02-2023|1120000|      11000|   1100000|
|         2|     samsung|01-03-2023|1080000|      11000|   1100000|
|         2|     samsung|01-04-2023|1800000|      11000|   1100000|
|         2|     samsung|01-05-2023| 980000|      11000|   1100000|
|         2|     samsung|01-06-2023|1100000|    

In [0]:
# question - send a mail to employee who all have not completed 8 hours in office when they comes to office
emp_data = [(1,"manish","11-07-2023","10:20"),
        (1,"manish","11-07-2023","11:20"),
        (2,"rajesh","11-07-2023","11:20"),
        (1,"manish","11-07-2023","11:50"),
        (2,"rajesh","11-07-2023","13:20"),
        (1,"manish","11-07-2023","19:20"),
        (2,"rajesh","11-07-2023","17:20"),
        (1,"manish","12-07-2023","10:32"),
        (1,"manish","12-07-2023","12:20"),
        (3,"vikash","12-07-2023","09:12"),
        (1,"manish","12-07-2023","16:23"),
        (3,"vikash","12-07-2023","18:08")]

emp_schema = ["id", "name", "date", "time"]
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

emp_df.show()

+---+------+----------+-----+
| id|  name|      date| time|
+---+------+----------+-----+
|  1|manish|11-07-2023|10:20|
|  1|manish|11-07-2023|11:20|
|  2|rajesh|11-07-2023|11:20|
|  1|manish|11-07-2023|11:50|
|  2|rajesh|11-07-2023|13:20|
|  1|manish|11-07-2023|19:20|
|  2|rajesh|11-07-2023|17:20|
|  1|manish|12-07-2023|10:32|
|  1|manish|12-07-2023|12:20|
|  3|vikash|12-07-2023|09:12|
|  1|manish|12-07-2023|16:23|
|  3|vikash|12-07-2023|18:08|
+---+------+----------+-----+



In [0]:
emp_df = emp_df.withColumn("timestamp",
                           from_unixtime(unix_timestamp(expr("CONCAT(date,' ',time)"),"dd-MM-yyyy HH:mm")))

In [0]:
emp_df.show()

+---+------+----------+-----+-------------------+
| id|  name|      date| time|          timestamp|
+---+------+----------+-----+-------------------+
|  1|manish|11-07-2023|10:20|2023-07-11 10:20:00|
|  1|manish|11-07-2023|11:20|2023-07-11 11:20:00|
|  2|rajesh|11-07-2023|11:20|2023-07-11 11:20:00|
|  1|manish|11-07-2023|11:50|2023-07-11 11:50:00|
|  2|rajesh|11-07-2023|13:20|2023-07-11 13:20:00|
|  1|manish|11-07-2023|19:20|2023-07-11 19:20:00|
|  2|rajesh|11-07-2023|17:20|2023-07-11 17:20:00|
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|
|  3|vikash|12-07-2023|09:12|2023-07-12 09:12:00|
|  1|manish|12-07-2023|16:23|2023-07-12 16:23:00|
|  3|vikash|12-07-2023|18:08|2023-07-12 18:08:00|
+---+------+----------+-----+-------------------+



In [0]:
window = Window.partitionBy("id","date").orderBy("date").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

In [0]:
new_df = emp_df.withColumn("login",first("timestamp").over(window))\
    .withColumn("logout",last("timestamp").over(window))\
        .withColumn("login",to_timestamp("login","yyyy-MM-dd HH:mm:ss"))\
            .withColumn("logout",to_timestamp("logout","yyyy-MM-dd HH:mm:ss"))\
                .withColumn("total_time",col("logout")-col("login")).show()

+---+------+----------+-----+-------------------+-------------------+-------------------+--------------------+
| id|  name|      date| time|          timestamp|              login|             logout|          total_time|
+---+------+----------+-----+-------------------+-------------------+-------------------+--------------------+
|  1|manish|11-07-2023|10:20|2023-07-11 10:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|11:20|2023-07-11 11:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|11:50|2023-07-11 11:50:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|11-07-2023|19:20|2023-07-11 19:20:00|2023-07-11 10:20:00|2023-07-11 19:20:00|INTERVAL '0 09:00...|
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|2023-07-12 10:32:00|2023-07-12 16:23:00|INTERVAL '0 05:51...|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|2023-07-12 10:32:00|2023-07-12 16:23:00|INTERVAL '0 05:51...|
|

### Running sales/ cumulative sales

In [0]:
# question - Find out performance of the sales based on last 3 months average

product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema)

In [0]:
product_df.show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         1|      iphone|01-01-2023|1500000|
|         2|     samsung|01-01-2023|1100000|
|         3|     oneplus|01-01-2023|1100000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



In [0]:
window = Window.partitionBy("product_id").orderBy("sales_date").rowsBetween(-2,0)
window1 = Window.partitionBy("product_id").orderBy("sales_date")

In [0]:
product_df.withColumn("running_sum",sum("sales").over(window))\
    .withColumn("row_number",row_number().over(window1))\
    .filter(col('row_number')>2)\
    .withColumn("average_sales",round((col("running_sum")/3),2))\
    .show()

+----------+------------+----------+-------+-----------+----------+-------------+
|product_id|product_name|sales_date|  sales|running_sum|row_number|average_sales|
+----------+------------+----------+-------+-----------+----------+-------------+
|         1|      iphone|01-03-2023|1600000|    4400000|         3|   1466666.67|
|         1|      iphone|01-04-2023|1700000|    4600000|         4|   1533333.33|
|         1|      iphone|01-05-2023|1200000|    4500000|         5|    1500000.0|
|         1|      iphone|01-06-2023|1100000|    4000000|         6|   1333333.33|
|         2|     samsung|01-03-2023|1080000|    3300000|         3|    1100000.0|
|         2|     samsung|01-04-2023|1800000|    4000000|         4|   1333333.33|
|         2|     samsung|01-05-2023| 980000|    3860000|         5|   1286666.67|
|         2|     samsung|01-06-2023|1100000|    3880000|         6|   1293333.33|
|         3|     oneplus|01-03-2023|1160000|    3380000|         3|   1126666.67|
|         3|    

### Rows Between

In [0]:

product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema)

In [0]:
window_rows = Window.partitionBy("product_id").orderBy("sales_date").rowsBetween(-2,0)


In [0]:
# window_range = Window.partitionBy("product_id").orderBy("sales_date").rangeBetween(Window.currentRow, 1)


In [0]:
product_df.withColumn("running_sum_rows", sum("sales").over(window_rows)).show()

+----------+------------+----------+-------+----------------+
|product_id|product_name|sales_date|  sales|running_sum_rows|
+----------+------------+----------+-------+----------------+
|         1|      iphone|01-01-2023|1500000|         1500000|
|         1|      iphone|01-02-2023|1300000|         2800000|
|         1|      iphone|01-03-2023|1600000|         4400000|
|         1|      iphone|01-04-2023|1700000|         4600000|
|         1|      iphone|01-05-2023|1200000|         4500000|
|         1|      iphone|01-06-2023|1100000|         4000000|
|         2|     samsung|01-01-2023|1100000|         1100000|
|         2|     samsung|01-02-2023|1120000|         2220000|
|         2|     samsung|01-03-2023|1080000|         3300000|
|         2|     samsung|01-04-2023|1800000|         4000000|
|         2|     samsung|01-05-2023| 980000|         3860000|
|         2|     samsung|01-06-2023|1100000|         3880000|
|         3|     oneplus|01-01-2023|1100000|         1100000|
|       

In [0]:
product_df.printSchema()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales_date: string (nullable = true)
 |-- sales: long (nullable = true)



In [0]:
from pyspark.sql import functions as F
# Convert 'sales_date' from string to date
product_df = product_df.withColumn("sales_date", F.to_date(F.col("sales_date"), "dd-MM-yyyy"))


In [0]:
# Define a window with ROWS between the current row and the next row
window_range = Window.partitionBy("product_id").orderBy("sales_date").rowsBetween(Window.currentRow, 1)


In [0]:
product_df.withColumn("running_sum_rows", sum("sales").over(window_range)).show()


# Calculate the running sum of 'sales' for each product over the specified window
# product_df.withColumn("running_sum_rows", F.sum("sales").over(window_range)).show()

+----------+------------+----------+-------+----------------+
|product_id|product_name|sales_date|  sales|running_sum_rows|
+----------+------------+----------+-------+----------------+
|         1|      iphone|2023-01-01|1500000|         2800000|
|         1|      iphone|2023-02-01|1300000|         2900000|
|         1|      iphone|2023-03-01|1600000|         3300000|
|         1|      iphone|2023-04-01|1700000|         2900000|
|         1|      iphone|2023-05-01|1200000|         2300000|
|         1|      iphone|2023-06-01|1100000|         1100000|
|         2|     samsung|2023-01-01|1100000|         2220000|
|         2|     samsung|2023-02-01|1120000|         2200000|
|         2|     samsung|2023-03-01|1080000|         2880000|
|         2|     samsung|2023-04-01|1800000|         2780000|
|         2|     samsung|2023-05-01| 980000|         2080000|
|         2|     samsung|2023-06-01|1100000|         1100000|
|         3|     oneplus|2023-01-01|1100000|         2220000|
|       